In [30]:
import json
import re
def read_abis(abi_path):
    abis = {}
    
    with open(abi_path, 'r', encoding='utf-8') as f:
        data:dict[str, dict[str, dict]] = json.load(f)

    for filename, contracts in data['contracts'].items():
        if filename not in abis:
            abis[filename] = {}

        for contract, details in contracts.items():
            if contract not in abis[filename]:
                abis[filename][contract] = {}
                
            for abi_item in details['abi']:
                
                abi_type = abi_item.get('type', None)
                abi_name = abi_item.get('name', None)
                abi_inputs = abi_item.get('inputs', None)
                
                if abi_type not in abis[filename]:
                    abis[filename][contract][abi_type] = {}
                
                if abi_type == 'constructor':
                    abi_name = 'constructor'
                
                if abi_type == 'fallback':
                    abi_name = 'fallback'
                
                if abi_name:
                    abis[filename][contract][abi_type][abi_name] = {}
                
                if abi_inputs:
                    for input_item in abi_inputs:
                        input_type = input_item.get('type', None)
                        input_name = input_item.get('name', None)
                        
                        if input_type not in abis[filename][contract][abi_type][abi_name]:
                            abis[filename][contract][abi_type][abi_name][input_type] = []
                        
                        if input_name:
                            abis[filename][contract][abi_type][abi_name][input_type].append(input_name)
    
    return abis

In [50]:
def file_import(path, importlists:list=None, G=None):
    if importlists is None:
        importlists = []
    if G is None:
        G = {}
    try:
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read()
        pattern = r"import\s+[\'\"](.+?)[\'\"]\;"
        matches = re.findall(pattern, content)
        file_name = os.path.basename(path)
        if file_name not in G:
            G[file_name] = []
        for match in matches:
            if match not in G[file_name]:
                G[file_name].append(match)
            matchpath = os.path.join(os.path.dirname(path), match[2:])
            if os.path.exists(matchpath):
                file_import(matchpath, importlists, G)
                importlists.append(matchpath)

    except Exception as e:
        pass
    return G, importlists

In [74]:
import re

filename = "integration_GnosisSafeProxy.sol"
filepath = "/home/lxm/solidity/solidity/new/codes/0x3cE9Bb52894E2d4Bc3B659B4F7a35f7556cB9FdD_GnosisSafeProxy/integration_GnosisSafeProxy.sol"

abis = read_abis("/home/lxm/solidity/solidity/new/jsons/0x3cE9Bb52894E2d4Bc3B659B4F7a35f7556cB9FdD_GnosisSafeProxy_type1.json")

G, importlists = file_import(filepath)

# scopes = find_scopes(filepath)
    

In [75]:
print(abis)

{'integration_GnosisSafeProxy.sol': {'GnosisSafeProxy': {'constructor': {'constructor': {'address': ['_singleton']}}, 'fallback': {'fallback': {}}}, 'GnosisSafeProxyFactory': {'event': {'ProxyCreation': {'address': ['proxy', 'singleton']}}, 'function': {'proxyRuntimeCode': {}}}, 'IProxy': {'function': {'masterCopy': {}}}, 'IProxyCreationCallback': {'function': {'proxyCreated': {'address': ['proxy', '_singleton'], 'bytes': ['initializer'], 'uint256': ['saltNonce']}}}}}


In [62]:
# TODO:补全数据类型


def get_parameters(parameters):
    parameters = parameters.split(',')
    parameters = [parameter.strip() for parameter in parameters]
    result = {}
    pattern = r"import\s+[\'\"](.+?)[\'\"]\;"
    pattern_1 = r"\s*(address|string|bool)\s*"
    pattern_2 = r"\s*(byte|uint)\s*"
    
    for parameter in parameters:
        parameter = parameter.split(' ')
        parameter_type = parameter[0]
        parameter_name = parameter[-1]
        if parameter_type not in result:
            result[parameter_type] = []
        result[parameter_type].append(parameter_name)
    return result

In [63]:
import os
def extract_functions(filepath, extracted=None):
    def find_scopes(filename):
        with open(filename, 'r') as f:
            contract_content = f.read()
        
        scopes = {'interface': {}, 'contract': {}, 'library': {}}
        current_scope = None
        current_name = None
        lines = contract_content.split('\n')
        stack = []
        for line in lines:
            # print(line)
            # print(stack)
            interface_match = re.match(r'^\s*interface\s+(\w+)\s*{', line)
            contract_match = re.match(r'^(abstract)*\s*contract\s+(\w+)\s*{', line)
            library_match = re.match(r'^\s*library\s+(\w+)\s*{', line)
            if interface_match:
                stack.append('{')
                # print(stack)
                current_scope = 'interface'
                current_name = interface_match.group(1)
                scopes[current_scope][current_name] = []
                continue
            if contract_match:
                stack.append('{')
                # print(stack)
                current_scope = 'contract'
                current_name = contract_match.group(2)
                scopes[current_scope][current_name] = []
                continue
                
            if library_match:
                stack.append('{')
                # print(stack)
                current_scope = 'library'
                current_name = library_match.group(1)
                scopes[current_scope][current_name] = []
                continue
            
            if '{' in line:
            # if re.match(r'*{*', line):
                stack.append('{')
                # print(stack)
                scopes[current_scope][current_name].append(line)
                continue
                
            if re.match(r'^\s*}\s*$', line):
                stack.pop()
                # print(stack)
                if len(stack) == 0:
                    current_scope = None
                    current_name = None
                else:
                    scopes[current_scope][current_name].append(line)
                continue
                
            if current_scope and current_name:
                scopes[current_scope][current_name].append(line)
        return scopes
    events_functions = find_scopes(filepath)
    filename = os.path.basename(filepath)
    extracted[filename] = {}
    for _, items in events_functions.items():
        for contract_name, code in items.items():
            extracted[filename][contract_name] = {}
            stack = []
            for index in range(len(code)):
                line = code[index]
                match = re.match(r'^\s*(event|function|error|modifier)\s+(\w+)\s*\((.*?)\)\s*', line)
                if match:
                    function_type = match.group(1)
                    if function_type not in extracted[filename][contract_name]:
                        extracted[filename][contract_name][function_type] = {}
                    function_name = match.group(2)
                    parameters = match.group(3)
                    
                    parameters = get_parameters(parameters)
                    
                    extracted[filename][contract_name][function_type][function_name] = parameters
                    continue
                match = re.match(r'^\s*(constructor|fallback)\s*\((.*?)\)\s*', line)
                if match:
                    function_type = match.group(1)
                    if function_type not in extracted[filename][contract_name]:
                        extracted[filename][contract_name][function_type] = {}
                    function_name = match.group(1)
                    parameters = match.group(2)
                    parameters = get_parameters(parameters)
                    extracted[filename][contract_name][function_type][function_name] = parameters
                    continue
                match = re.match(r'^\s*(event|function|error|modifier)\s+(\w+)\s*\(*', line)
                if match:
                    stack.append('(')
                    function_type = match.group(1)
                    if function_type not in extracted[filename][contract_name]:
                        extracted[filename][contract_name][function_type] = {}
                    function_name = match.group(2)
                    parameters = line
                    index += 1
                    while index < len(code):
                        line = code[index]
                        if ')' in line:
                            stack.pop()
                            parameters += line.strip()
                            
                            if len(stack) == 0:
                                parameters = re.match(r'^\s*(event|function|error|modifier)\s+(\w+)\s*\((.*?)\)\s*', parameters).group(3)
                                parameters = get_parameters(parameters)
                                extracted[filename][contract_name][function_type][function_name] = parameters
                                break
                        parameters += line.strip()
                        index += 1
                match = re.match(r'^\s*(constructor|fallback)\s+(\w+)\s*\(*', line)
                if match:
                    stack.append('(')
                    function_type = match.group(1)
                    if function_type not in extracted[filename][contract_name]:
                        extracted[filename][contract_name][function_type] = {}
                    function_name = match.group(2)
                    parameters = line
                    index += 1
                    while index < len(code):
                        line = code[index]
                        if ')' in line:
                            stack.pop()
                            parameters += line.strip()
                            
                            if len(stack) == 0:
                                parameters = re.match(r'^\s*(constructor|fallback)\s*\((.*?)\)\s*', parameters).group(2)
                                parameters = get_parameters(parameters)
                                extracted[filename][contract_name][function_type][function_name] = parameters
                                break
                        parameters += line.strip()
                        index += 1
    return extracted

extracted = {}

functions_events = extract_functions(filepath, extracted)



In [78]:
for filename, items in functions_events.items():
    print(f"{filename}:")
    for contract_name, functions in items.items():
        print(f"{contract_name}:")
        for function_type, functions in functions.items():
            print(f"  {function_type}:")
            for function_name, parameters in functions.items():
                print(f"    {function_name}")
                for parameters_type, parameters_name in parameters.items():
                    print(f"      {parameters_type}: {parameters_name}")
                # parameters = get_parameters(parameters)
                # print(parameters)
            # print()

integration_GnosisSafeProxy.sol:
IProxy:
  function:
    masterCopy
      : ['']
IProxyCreationCallback:
  function:
    proxyCreated
      GnosisSafeProxy: ['proxy']
      address: ['_singleton']
      bytes: ['initializer']
      uint256: ['saltNonce']
GnosisSafeProxy:
  constructor:
    constructor
      address: ['_singleton']
  fallback:
    fallback
      : ['']
GnosisSafeProxyFactory:
  event:
    ProxyCreation
      GnosisSafeProxy: ['proxy']
      address: ['singleton']
  function:
    createProxy
      address: ['singleton']
      bytes: ['data']
    proxyRuntimeCode
      : ['']
    proxyCreationCode
      : ['']
    deployProxyWithNonce
      address: ['_singleton']
      bytes: ['initializer']
      uint256: ['saltNonce']
    createProxyWithNonce
      address: ['_singleton']
      bytes: ['initializer']
      uint256: ['saltNonce']
    createProxyWithCallback
      address: ['_singleton']
      bytes: ['initializer']
      uint256: ['saltNonce']
      IProxyCreationCallba

In [59]:
basefilepath = os.path.dirname(filepath)
imports = []
if filename not in G:
    imports = G[filename]
    print(imports)
if imports:
    print(1)
    for import_file in imports:
        extracted[import_file] = extract_functions(os.path.join(basefilepath,import_file), extracted)


In [81]:
# TODO:补全相关比较
for filename, items in functions_events.items():
    print(f"{filename}:")
    if filename not in abis:
        print(1)
        continue
    for contract_name, functions in items.items():
        print(f"{contract_name}:")
        if contract_name not in abis[filename]:
            print(2)
            continue
        for function_type, functions in functions.items():
            print(f"  {function_type}:")
            if function_type not in abis[filename][contract_name]:
                print(3)
                continue
            for function_name, parameters in functions.items():
                print(f"    {function_name}")
                if function_name not in abis[filename][contract_name][function_type]:
                    print(4)
                    continue
                for parameters_type, parameters_names in parameters.items():
                    if parameters_type:
                        print(f"      {parameters_type}: {parameters_name}")
                        print(parameters_type)
                        if parameters_type not in abis[filename][contract_name][function_type][function_name]:
                            print(5)
                            continue
                        for parameter_name in parameters_names:
                            if parameter_name not in abis[filename][contract_name][function_type][function_name][parameters_type]:
                                print(6)

integration_GnosisSafeProxy.sol:
IProxy:
  function:
    masterCopy
IProxyCreationCallback:
  function:
    proxyCreated
      GnosisSafeProxy: ['saltNonce']
GnosisSafeProxy
5
      address: ['saltNonce']
address
      bytes: ['saltNonce']
bytes
      uint256: ['saltNonce']
uint256
GnosisSafeProxy:
  constructor:
    constructor
      address: ['saltNonce']
address
  fallback:
    fallback
GnosisSafeProxyFactory:
  event:
    ProxyCreation
      GnosisSafeProxy: ['saltNonce']
GnosisSafeProxy
5
      address: ['saltNonce']
address
  function:
    createProxy
4
    proxyRuntimeCode
    proxyCreationCode
4
    deployProxyWithNonce
4
    createProxyWithNonce
4
    createProxyWithCallback
4
    calculateCreateProxyWithNonceAddress
4
